In [5]:
from mtcnn import MTCNN
import cv2
import matplotlib.pyplot as plt
import time
from keras_facenet import FaceNet
from numpy import asarray
import pickle
import numpy as np

# Extracting features from the cropped images

## Classes to hold the images and their information 

In [6]:
class Image:
    def __init__ (self, name , img):
        self.name = name
        self.img=img

In [7]:
class ProcessedImage:
    def __init__ (self, name , vector):
        self.name = name
        self.vector=vector

## Utility class

In [8]:
def ProcessName(name):
    x=name.split()
    return x[0]

In [9]:
#Read image into a list
import os

def load_images_from_folder(folder):
    images = []
    for subFolder in os.listdir(folder):
        ExactSubFolder = os.path.join(folder,subFolder)
        for filename in os.listdir(ExactSubFolder):
            Temp=os.path.join(folder,subFolder)
            img = cv2.cvtColor(cv2.imread(os.path.join(Temp,filename)), cv2.COLOR_BGR2RGB)
            if img is not None:
               images.append(Image(ProcessName(filename),img))
    return images




## Loading images into a list

In [14]:
img_List=load_images_from_folder('./Train')  #preferbly pictures that only have 1 person in 'em

## Cropping detected faces from the image list and extracting features from them 

In [15]:
embedder = FaceNet()
detector = MTCNN(min_face_size=200)

Data=[]
for img in img_List:
    detections = detector.detect_faces(img.img)

    #bounding box
    img_crop = []
    for det in detections:
        #cv2.rectangle(img, (det['box'][0],det['box'][1]), (det['box'][0]+det['box'][2], det['box'][1]+det['box'][3]), (0, 255, 0), 2)
        img_crop.append(img.img[det['box'][1]:det['box'][1]+det['box'][3], det['box'][0]:det['box'][0]+det['box'][2]])
    
    if (len(img_crop) == 1 ):
        #extract features
        new_trainx =list()
        
        for cropped in img_crop:
            embedding = embedder.embeddings(img_crop)
            new_trainx.append(embedding)

        new_trainx = asarray(new_trainx)
        Data.append(ProcessedImage(img.name,new_trainx[0][0]))

## Serializing feature vector + name

In [16]:
with open('Extracted_Features', 'wb') as f:
        pickle.dump(Data, f)